In [1]:
import requests
import pandas as pd
import numpy as np
from elasticsearch import Elasticsearch
import os
from datetime import datetime as dt
import os.path
from os import path
import sys
from importlib import reload
import copy
import pickle

In [2]:
sys.path.append('..')

In [3]:
import modules.Elastic_Module
reload(modules.Elastic_Module)

<module 'modules.Elastic_Module' from '../modules/Elastic_Module.py'>

In [4]:
from modules.Elastic_Module import *

In [5]:
url_ckan='http://datos.gob.cl/api/3/action/package_list'
url_package='http://datos.gob.cl/api/3/action/package_show?id='
url_package_search='http://datos.gob.cl/api/3/action/package_search?'

In [6]:
#Listas de paquetes ckan (con sus ID)
ckan_packages=requests.get(url_ckan) #consultar api
ckan_packages=ckan_packages.json() #convertir respuesta a json

In [7]:
#OBTENER TODOS LOS PAQUETES DE LA BASE DE DATOS CKAN
rows=str(1000) #maximo numero de resultados por consulta (maximo permitido es 1000)
packs=[] #contenedor de paquetes retornados
for s in range(0,4):
    start=str(s*int(rows)) #paginacion
    packages=requests.get(url_package_search+'rows='+rows+'&'+'start='+start)
    packages=packages.json()
    packs.extend(packages['result']['results'])

In [8]:
recursos=[]
for p in packs:
    for r in p['resources']:
        d=copy.deepcopy(r)
        #parámetros del paquete
        d['package_id']=p['id'] #id
        d['package_name']=p['name'] #nombre
        d['package_title']=p['title'] #titulo
        d['package_notes']=p['notes'] #notas
        if 'organization' in p.keys(): #si es que existe info de organizacion
            if p['organization']!=None:
                d['inst.']=p['organization']['title'] #si es que existe info del titulo incluirlo
                d['inst_name']=p['organization']['name'] #si es que existe info del titulo incluirlo
                d['inst_id']=p['organization']['id'] #si es que existe info del id inst.
            else:
                d['inst.']=None
                d['inst_name']=None
                d['inst_id']=None
        recursos.append(d)
recursos=pd.DataFrame(recursos) #convertir a df      

In [9]:
recursos=recursos.rename(columns={'last_modified':'fecha','format':'formato'}) #cambiar nombres de columnas

In [11]:
recursos=recursos.drop(['mimetype','cache_url','url','cache_last_updated','hash','url_type',
              'mimetype_inner','position','resource_type'],axis=1) #botar columnas

In [12]:
recursos['fecha']=pd.to_datetime(recursos['fecha']) #convertir la fecha a formato solo con la fecha (sin la hora)
#desagregar en dia, mes, año
recursos['dia']=recursos['fecha'].dt.date
recursos['year']=recursos['fecha'].dt.year
recursos['mes']=recursos['fecha'].dt.month

In [11]:
data=recursos.to_dict(orient='records')

In [22]:
#guardar paso en csv
path='../ETLcache/'
now = dt.now()
date_time = now.strftime("_%m-%d-%Y_%H-%M-%S")

In [23]:
recursos.to_csv(path+date_time+'.csv')

In [24]:
#guardar data de paquetes cruda en pickle y borrar para liberar espacio en memoria
with open(path+date_time+'.pickle', 'wb') as f:
    pickle.dump(packs, f)
del(packs)

In [13]:
#es = Elasticsearch(hosts = [ES_HOST],http_auth=('elastic', '                    # replace with the empty string'))

In [27]:
#variables de entorno
args={}
args['elastic_user']='elastic'
args['elastic_psswd']=os.environ['ELASTIC_PASSWD']
args['DOMAIN_ELASTIC']=os.environ['DOMAIN_ELASTIC']
args['ELASTIC_PORT']=os.environ['ELASTIC_PORT']

In [28]:
ES_HOST = "https://"+args['DOMAIN_ELASTIC']
basic_auth={'user':args['elastic_user'], 'psswd':args['elastic_psswd']}

In [42]:
es=elastic_connection(ES_HOST,basic_auth=basic_auth)

In [43]:
es.ping()

True


In [44]:
#generar campos concatenados (parámetros)
generateID_params_custom={'fields':['package_id','id'], #campos a concatenar
                               'head':20, #numero de caracteres a dejar
                                'sep':'_' #separador entre campos
                               }

In [45]:
es.generateID_params_custom=generateID_params_custom

In [46]:
es.add_docs(recursos)

In [47]:
#prepar la data para elastic search
es.elastic_prepare_pattern()

In [48]:
indexes=[
    ('ckan','ckan')
        ]

In [49]:
#generar bulk data
i=indexes[0]
n=1500
id_field='id'
es.bulk_data(INDEX_NAME=i[0],_type=i[1],n=n,id_field=id_field)
#test

creando paquetes de data bulk
size of bulk docs is : 0.1796875 Kbs
N° docs: 10871.0, N° chunks: 15


In [50]:
len(es.chunks[0])

1500

In [53]:
es.clear_index(i[0])

deleting 'ckan' index...
 response: '{'acknowledged': True}'
creating 'ckan' index...
 response: '{'acknowledged': True, 'shards_acknowledged': True, 'index': 'ckan'}'


In [54]:
es.bulk_index()

bulk indexing...:index---> ckan
 indexing chunk : 14 of 15finished bulk indexing
No recorded ES indexing errors
No recorded ES indexing exceptions


In [57]:
es.errors

AttributeError: 'elastic_connection' object has no attribute 'errors'

In [55]:
es.test_index(i[0],n=15)

../modules/Elastic_Module.py:78: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  df = json_normalize([x['_source'] for x in res['hits']['hits']])


,description,name,created,datastore_active,formato,package_id,state,fecha,revision_id,datastore_contains_all_records_of_source_file,...,set_url_type,ignore_hash,ckan_url,task_created,document_image_upload,document_clear_upload,document_image_url,dia,year,mes
0,Indemnizados del Carbón 2019-INDEMNIZADOS DEL ...,Indemnizados del Carbón 2019,2020-05-22T17:42:58.533295,True,XLSX,e228645e-b207-4a74-b750-462637e8ce46,active,2020-05-22T20:42:58.464441,d10e3c3a-7c74-4dbb-a60c-a6247d1432a2,True,...,False,True,https://prod.datos.gob.cl,2020-11-10 00:27:15.714119,None,None,None,2020-05-22,2020.0,5.0
1,RESUMEN ANUAL BONIFICACION AL INGRESO ETICO FA...,IEF 2018,2020-05-22T17:34:22.464561,True,CSV,70d50ce9-470f-43c4-a9be-0ed902b680b5,active,2020-05-22T20:34:22.364282,70f9bbc7-90d2-4d73-a394-9cae55852f21,True,...,False,True,https://prod.datos.gob.cl,2020-11-10 00:25:57.917088,None,None,None,2020-05-22,2020.0,5.0
2,RESUMEN ANUAL BONIFICACION AL INGRESO ETICO FA...,IEF 2018,2020-05-22T17:34:50.361617,True,XLSX,70d50ce9-470f-43c4-a9be-0ed902b680b5,active,2020-05-22T20:34:50.287459,5424cbb6-abb5-46de-9818-ea15bd46f002,True,...,False,True,https://prod.datos.gob.cl,2020-11-10 00:26:00.648670,None,None,None,2020-05-22,2020.0,5.0
3,RESUMEN ANUAL BONIFICACION AL INGRESO ETICO FA...,IEF 2019,2020-05-22T17:35:15.845787,True,CSV,70d50ce9-470f-43c4-a9be-0ed902b680b5,active,2020-05-22T20:35:54.234423,0af36f46-2bbd-494a-ac9f-e69c5d9fb010,True,...,False,True,https://prod.datos.gob.cl,2020-11-10 00:26:03.353932,None,None,None,2020-05-22,2020.0,5.0
4,RESUMEN ANUAL BONIFICACION AL INGRESO ETICO FA...,IEF 2019,2020-05-22T17:36:50.986275,True,XLSX,70d50ce9-470f-43c4-a9be-0ed902b680b5,active,2020-05-22T20:36:50.912384,0af36f46-2bbd-494a-ac9f-e69c5d9fb010,True,...,False,True,https://prod.datos.gob.cl,2020-11-10 00:26:06.067385,None,None,None,2020-05-22,2020.0,5.0
5,"RESUMEN ANUAL BONO PROTECCIÓN IEF POR MES, TIP...",BPIEF 2018,2020-05-22T17:22:47.319900,True,CSV,526b468c-4939-4a7c-b1bb-43d1faa5b9b1,active,2020-05-22T20:22:47.233086,ae630bf2-9656-4218-a2b6-684de498c5ef,True,...,False,True,https://prod.datos.gob.cl,2020-11-09 23:57:06.903779,None,None,None,2020-05-22,2020.0,5.0
6,"RESUMEN ANUAL BONO PROTECCIÓN IEF POR MES, TIP...",BPIEF 2018,2020-05-22T17:23:06.681920,True,XLSX,526b468c-4939-4a7c-b1bb-43d1faa5b9b1,active,2020-05-22T20:23:06.612508,270a1acb-7f96-4e33-9c09-fb24dea249d9,True,...,False,True,https://prod.datos.gob.cl,2020-11-09 23:57:11.458486,None,None,None,2020-05-22,2020.0,5.0
7,"RESUMEN ANUAL BONO PROTECCIÓN IEF POR MES, TIP...",BPIEF 2019,2020-05-22T17:23:24.883566,True,CSV,526b468c-4939-4a7c-b1bb-43d1faa5b9b1,active,2020-05-22T20:23:24.770043,7bdb932b-9c97-480d-af03-40e7fd86a0d6,True,...,False,True,https://prod.datos.gob.cl,2020-11-09 23:57:11.488773,None,None,None,2020-05-22,2020.0,5.0
8,"RESUMEN ANUAL BONO PROTECCIÓN IEF POR MES, TIP...",BPIEF 2019,2020-05-22T17:23:42.039276,True,XLSX,526b468c-4939-4a7c-b1bb-43d1faa5b9b1,active,2020-05-22T20:23:41.972252,4ee220fb-9c84-4c6b-adb3-7ee0d8f9c7a8,True,...,False,True,https://prod.datos.gob.cl,2020-11-09 23:57:11.518227,None,None,None,2020-05-22,2020.0,5.0
9,Bono Bodas de Oro - 2018 -RESUMEN ANUAL BONO B...,BDO 2018,2020-05-22T17:17:40.180401,True,CSV,05826b0c-cac2-4f70-a054-2d4813a78561,active,2020-05-22T20:17:40.118237,9cc91e25-f4a4-4060-979d-39a2c5689a03,True,...,False,True,https://prod.datos.gob.cl,2020-11-09 23:57:06.728246,None,None,None,2020-05-22,2020.0,5.0


In [56]:
es.list_indices()

['ckan', '.security-7', '.kibana_1']